In [3]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

hunt_file = 'Data\\Hunt\\J_A+A_673_A114_clusters.dat.gz.fits'
kharchenko_file = 'Data\\Kharchenko\\J_A+A_558_A53_catalog.dat.fits'

def datahandler(filename):
    hdul = fits.open(filename)
    hdr1 = hdul[0].header #first and second header
    hdr2 = hdul[1].header
    data, cols = hdul[1].data, hdul[1].columns
    print('Some metadata')
    for i, j in enumerate(cols.names): #printing metedata
        print(hdr2[f"TTYPE{i+1}"],"-->", hdr2.comments[f"TTYPE{i+1}"])
    print("""
    """)
    return hdul[1].data, hdul[1].columns

data_hunt, cols_hunt = datahandler(hunt_file)
data_khar, cols_khar = datahandler(kharchenko_file)

Some metadata
Name --> Main accepted cluster name (1)
ID --> ? Internal cluster ID
AllNames --> Comma separated list of all crossmatched
Type --> [omg] Type of object (o: open cluster, m:
CST --> [3/99] Astrometric SNR (cluster significance
N --> [10/153797] Number of member stars
CSTt --> [2.9/99] Astrometric SNR within tidal radius
Nt --> [10/53464] Number of stars within tidal radius
RAdeg --> Right ascension of densest point (ICRS) at
DEdeg --> Declination of densest point (ICRS) at
GLON --> Galactic longitude
GLAT --> Galactic latitude
r50 --> Radius containing 50% of members within the
rc --> Core radius (approximate estimate)
rt --> Tidal radius (approximate estimate)
rtot --> Total radius (including tidal tails, coma, etc)
r50pc --> Radius containing 50% of members in parsecs
rcpc --> Core radius in parsecs
rtpc --> Tidal radius in parsecs
rtotpc --> Total radius in parsecs
pmRA --> Mean proper motion in right ascension multipled
s_pmRA --> Standard deviation of pmRA
e_pmRA -->

In [5]:
print('stinkieeeeee')

stinkieeeeee
